## Day 24 Lecture 2 Assignment

In this assignment, we will build our a more complex logistic regression model, this time on both numeric and categorical data. We will use the Chicago traffic crashes dataset loaded below and analyze the model generated for this dataset.

In [33]:
%matplotlib inline
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def missingness_summary(df, print_log=False, sort='none'):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])
    
    if print_log == True:
      if sort == 'none':
          print(summary)
      elif sort == 'ascending':
          print(summary.sort_values())
      elif sort == 'descending':
          print(summary.sort_values(ascending=False))
      else:
          print('Invalid value for sort parameter.')
        
    return summary

In [3]:
crash_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv')

In [4]:
crash_data.head()

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN
0,JC334993,7/4/2019 22:33,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",7/4/2019 23:05,FOLLOWING TOO CLOSELY,NOT APPLICABLE,300,N,LAKE SHORE DR SB,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC370822,7/30/2019 10:22,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",7/30/2019 10:25,FAILING TO YIELD RIGHT-OF-WAY,IMPROPER TURNING/NO SIGNAL,8201,S,DR MARTIN LUTHER KING JR DR,631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC387098,8/10/2019 17:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"$501 - $1,500",8/10/2019 17:35,EQUIPMENT - VEHICLE CONDITION,NOT APPLICABLE,6747,S,CREGIER AVE,332.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC395195,8/16/2019 16:53,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"$501 - $1,500",8/16/2019 16:53,UNABLE TO DETERMINE,NOT APPLICABLE,554,N,FRANKLIN ST,1831.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,JC396604,8/17/2019 16:04,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,PARKING LOT,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"$501 - $1,500",8/17/2019 18:30,UNABLE TO DETERMINE,UNABLE TO DETERMINE,3700,N,WESTERN AVE,1921.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0


First, create a binary response column by modifying the "DAMAGE" column. Consider "OVER \$1500" to be the positive class, and under \$1500 to be the negative class.

In [5]:
# answer goes here
crash_data['HIGH_DAMAGE'] = (crash_data['DAMAGE'] == 'OVER $1,500').astype(int)

Using the code from Day 21, Lecture 1 as a starting point, devise an appropriate way to address missing values. You have a lot of freedom here; we will proceed by taking the following steps:

- Dropping all columns with more than 5% missing data
- Imputing the median for numeric columns with less than 5% missing data (except for STREET_NO; imputing it in this manner would not make any sense)
- Dropping rows with missing data for categorical columns that have less than 5% missing data

In [6]:
# answer goes here
missing_report = missingness_summary(crash_data, print_log=True, sort='descending')

WORKERS_PRESENT_I                0.998352
DOORING_I                        0.996616
WORK_ZONE_TYPE                   0.994391
WORK_ZONE_I                      0.992933
PHOTOS_TAKEN_I                   0.987318
STATEMENTS_TAKEN_I               0.979760
NOT_RIGHT_OF_WAY_I               0.953917
INTERSECTION_RELATED_I           0.779457
HIT_AND_RUN_I                    0.722423
LANE_CNT                         0.467107
REPORT_TYPE                      0.023012
MOST_SEVERE_INJURY               0.005795
INJURIES_NO_INDICATION           0.005776
INJURIES_UNKNOWN                 0.005776
INJURIES_TOTAL                   0.005776
INJURIES_REPORTED_NOT_EVIDENT    0.005776
INJURIES_NON_INCAPACITATING      0.005776
INJURIES_INCAPACITATING          0.005776
INJURIES_FATAL                   0.005776
NUM_UNITS                        0.003755
BEAT_OF_OCCURRENCE               0.000011
STREET_DIRECTION                 0.000005
STREET_NAME                      0.000003
FIRST_CRASH_TYPE                 0

In [7]:
missing_data_columns = missing_report.loc[missing_report > 0.05].index
crash_data.drop(columns=missing_data_columns, inplace=True)

In [8]:
crash_num_df = crash_data.select_dtypes(include='number')
crash_cat_df = crash_data.select_dtypes(include='O')

In [9]:
for col in crash_num_df.columns:
  crash_data[col] = crash_data[col].fillna(crash_data[col].median())

In [10]:
crash_data.dropna(inplace=True)

In [11]:
missing_report = missingness_summary(crash_data, print_log=True, sort='descending')

HIGH_DAMAGE                      0.0
INJURIES_UNKNOWN                 0.0
CRASH_DATE                       0.0
POSTED_SPEED_LIMIT               0.0
TRAFFIC_CONTROL_DEVICE           0.0
DEVICE_CONDITION                 0.0
WEATHER_CONDITION                0.0
LIGHTING_CONDITION               0.0
FIRST_CRASH_TYPE                 0.0
TRAFFICWAY_TYPE                  0.0
ALIGNMENT                        0.0
ROADWAY_SURFACE_COND             0.0
ROAD_DEFECT                      0.0
REPORT_TYPE                      0.0
CRASH_TYPE                       0.0
DAMAGE                           0.0
DATE_POLICE_NOTIFIED             0.0
PRIM_CONTRIBUTORY_CAUSE          0.0
SEC_CONTRIBUTORY_CAUSE           0.0
STREET_NO                        0.0
STREET_DIRECTION                 0.0
STREET_NAME                      0.0
BEAT_OF_OCCURRENCE               0.0
NUM_UNITS                        0.0
MOST_SEVERE_INJURY               0.0
INJURIES_TOTAL                   0.0
INJURIES_FATAL                   0.0
I

Finally, choose a few numeric and categorical features (2-3 of each) to include in the model. (You can definitely include more than this, but too many features, especially categorical ones, will most likely lead to convergence issues). One hot encode the chosen categorical features, being sure to omit one of the categories (which will serve as a "reference" level) to avoid perfect multicollinearity.

Again, you have a lot of freedom here; we will proceed with the following features, dropping the most commonly occurring category for the two categorical variables ("CLEAR" for weather, "REAR END" for first crash type):
POSTED_SPEED_LIMIT, WEATHER_CONDITION, INJURIES_TOTAL, FIRST_CRASH_TYPE

In [12]:
crash_features = crash_data.filter(['POSTED_SPEED_LIMIT', 'INJURIES_TOTAL'])

In [13]:
# answer goes here
weather_df = pd.get_dummies(crash_data['WEATHER_CONDITION'], drop_first=True)
# crash_features.drop('WEATHER_CONDITION', axis=1, inplace=True)
crash_features = pd.concat([crash_features, weather_df], axis=1)


crashtype_df = pd.get_dummies(crash_data['FIRST_CRASH_TYPE'], drop_first=True)
# crash_features.drop('FIRST_CRASH_TYPE', axis=1, inplace=True)
crash_features = pd.concat([crash_features, crashtype_df], axis=1)

In [14]:
crash_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362483 entries, 6 to 372584
Data columns (total 29 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   POSTED_SPEED_LIMIT            362483 non-null  int64  
 1   INJURIES_TOTAL                362483 non-null  float64
 2   CLEAR                         362483 non-null  uint8  
 3   CLOUDY/OVERCAST               362483 non-null  uint8  
 4   FOG/SMOKE/HAZE                362483 non-null  uint8  
 5   FREEZING RAIN/DRIZZLE         362483 non-null  uint8  
 6   OTHER                         362483 non-null  uint8  
 7   RAIN                          362483 non-null  uint8  
 8   SEVERE CROSS WIND GATE        362483 non-null  uint8  
 9   SLEET/HAIL                    362483 non-null  uint8  
 10  SNOW                          362483 non-null  uint8  
 11  UNKNOWN                       362483 non-null  uint8  
 12  ANIMAL                        362483 non-nul

Split the data into train and test, with 80% training and 20% testing. By default, the LR output from statsmodels does not include an intercept terms; add a constant column to the training data so that an intercept term is calculated for the LR model (hint: sm.add_constant() is a useful function to accomplish this).

In [18]:
# answer goes here
X = crash_features
Y = crash_data['HIGH_DAMAGE']

X_train, x_test, Y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [20]:
X_train_const = sm.add_constant(X_train)

Fit the logistic regression model using the statsmodels package and print out the coefficient summary. Which variables (in particular, which categories of our categorical variables) appear to be the most important, and what effect do they have on the probability of a crash resulting in $1500 or more in damages?

In [32]:
# answer goes here
sm_model = sm.Logit(Y_train, X_train_const).fit()
sm_model.summary()

Optimization terminated successfully.
         Current function value: 0.657550
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            HIGH_DAMAGE   No. Observations:               289986
Model:                          Logit   Df Residuals:                   289956
Method:                           MLE   Df Model:                           29
Date:                Thu, 15 Oct 2020   Pseudo R-squ.:                 0.04056
Time:                        19:01:18   Log-Likelihood:            -1.9068e+05
converged:                       True   LL-Null:                   -1.9874e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            0.7994      0.563      1.421      0.155      -0.303       1.902
POSTED_SPEED_LIMIT               0.0108      0.001     18.370      0.000       0.010       0.012
INJURIES_TOTAL                   0.7391      0.012     62.097      0.000       0.716       0.762
CLEAR                           -0.4579      0.562     -0.814      0.415      -1.560       0.644
CLOUDY/OVERCAST                 -0.3491      0.563     -0.621      0.535      -1.452       0.754
FOG/SMOKE/HAZE                  -0.1987      0.569     -0.349      0.727      -1.315       0.917
FREEZING RAIN/DRIZZLE            0.0755      0.594      0.127      0.899      -1.088       1.239
OTHER                           -0.1950      0.566     -0.344      0.731      -1.305       0.915
RAIN                            -0.3199      0.562     -0.569      0.569      -1.422       0.782
SEVERE CROSS WIND GATE          -0.0596      0.631     -0.095      0.925      -1.296       1.177
SLEET/HAIL                      -0.2765      0.571     -0.484      0.628      -1.395       0.842
SNOW                            -0.3309      0.563     -0.588      0.556      -1.434       0.772
UNKNOWN                         -0.3086      0.562     -0.549      0.583      -1.411       0.794
ANIMAL                          -0.9155      0.151     -6.067      0.000      -1.211      -0.620
FIXED OBJECT                     0.0703      0.024      2.982      0.003       0.024       0.117
HEAD ON                         -0.0744      0.047     -1.581      0.114      -0.167       0.018
OTHER NONCOLLISION              -1.1407      0.065    -17.478      0.000      -1.269      -1.013
OTHER OBJECT                    -0.6399      0.042    -15.368      0.000      -0.722      -0.558
OVERTURNED                       0.8940      0.262      3.408      0.001       0.380       1.408
PARKED MOTOR VEHICLE            -0.4758      0.015    -31.471      0.000      -0.505      -0.446
PEDALCYCLIST                    -2.4790      0.040    -61.813      0.000      -2.558      -2.400
PEDESTRIAN                      -2.4098      0.032    -74.664      0.000      -2.473      -2.347
REAR END                        -0.6768      0.015    -45.658      0.000      -0.706      -0.648
REAR TO FRONT                   -0.7350      0.068    -10.744      0.000      -0.869      -0.601
REAR TO REAR                    -0.8932      0.139     -6.442      0.000      -1.165      -0.621
REAR TO SIDE                    -0.5782      0.085     -6.805      0.000      -0.745      -0.412
SIDESWIPE OPPOSITE DIRECTION    -0.6426      0.033    -19.289      0.000      -0.708      -0.577
SIDESWIPE SAME DIRECTION        -0.6545      0.016    -41.422      0.000      -0.685      -0.623
TRAIN                            1.5030      1.043      1.441      0.150      -0.541       3.547
TURNING                         -0.1867      0.017    -11.292      0.000      -0.219      -0.154
========================================

*Should have scaled/normalized the continuous variables first...*

But it appears that many of these features are significant. Some that are significant and have large coefficients from the categoricals are 'PEDESTRIAN', 'PEDALCYCLIST', 'TRAIN', and 'OTHER NONCOLLISION'

Both continuous variables are significant but on different scales.

Create a LogisticRegression model with sklearn. Use the .predict() method (using X_test) to get a y_pred. Create a confusion matrix comparing your actual y_test to your prediction. What do you notice about your type of error?

In [23]:
logit = LogisticRegression(max_iter=1000)

logit.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
logit.score(X_train, Y_train)

0.5843420027173726

In [25]:
logit.score(x_test, y_test)

0.5812654316730348

In [29]:
logit_params = pd.Series(logit.coef_.reshape(-1,), index= X_train.columns)

logit_params['intercept'] = logit.intercept_[0]

logit_params

POSTED_SPEED_LIMIT              0.010792
INJURIES_TOTAL                  0.736133
CLEAR                          -0.158544
CLOUDY/OVERCAST                -0.048864
FOG/SMOKE/HAZE                  0.104495
FREEZING RAIN/DRIZZLE           0.333596
OTHER                           0.107708
RAIN                           -0.020421
SEVERE CROSS WIND GATE          0.127816
SLEET/HAIL                      0.028418
SNOW                           -0.031283
UNKNOWN                        -0.011332
ANIMAL                         -0.922314
FIXED OBJECT                    0.065806
HEAD ON                        -0.077560
OTHER NONCOLLISION             -1.144516
OTHER OBJECT                   -0.642561
OVERTURNED                      0.583318
PARKED MOTOR VEHICLE           -0.478154
PEDALCYCLIST                   -2.467100
PEDESTRIAN                     -2.404785
REAR END                       -0.679220
REAR TO FRONT                  -0.723578
REAR TO REAR                   -0.948245
REAR TO SIDE    

In [31]:
np.abs(logit_params).sort_values(ascending=False)

PEDALCYCLIST                    2.467100
PEDESTRIAN                      2.404785
OTHER NONCOLLISION              1.144516
REAR TO REAR                    0.948245
ANIMAL                          0.922314
INJURIES_TOTAL                  0.736133
REAR TO FRONT                   0.723578
REAR END                        0.679220
SIDESWIPE SAME DIRECTION        0.657574
SIDESWIPE OPPOSITE DIRECTION    0.644727
OTHER OBJECT                    0.642561
OVERTURNED                      0.583318
REAR TO SIDE                    0.582886
intercept                       0.503478
PARKED MOTOR VEHICLE            0.478154
FREEZING RAIN/DRIZZLE           0.333596
TURNING                         0.188454
CLEAR                           0.158544
SEVERE CROSS WIND GATE          0.127816
TRAIN                           0.119407
OTHER                           0.107708
FOG/SMOKE/HAZE                  0.104495
HEAD ON                         0.077560
FIXED OBJECT                    0.065806
CLOUDY/OVERCAST 

In [35]:
confusion_matrix(y_test, logit.predict(x_test))
#[True Negative, False Positive/type I error],
#[False Negative/type II error, True Positive]

array([[ 9468, 22424],
       [ 7933, 32672]])

There is a large amount of Type I errors (False Positive) in the model. Clearly, the model isn't very accurate in predicting the outcome variable.